# ASEを使ってQE用の構造モデル（Graphite slab, Graphene）を作成する

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

In [ ]:
import importlib
if importlib.util.find_spec("ase") is None:
    !pip install git+https://github.com/minoru-otani/ase.git@qe_rism

In [ ]:
import copy
import matplotlib.pyplot as plt
from ase import Atoms
from ase.io import read, write
from ase.visualize import view
from ase.visualize.plot import plot_atoms
from ase.constraints import FixAtoms
from ase.io.cube import read_cube_data
from ase.build import fcc111, graphene, add_adsorbate, molecule, cut, add_vacuum
from ase.geometry import wrap_positions
from ase.lattice.hexagonal import Graphite, Graphene

## Grapheneを使ってスラブモデルを作成する

3-1で使った表面構造を作る関数を用いてGraphene

In [ ]:
from inspect import signature
signature(Graphite)

In [ ]:
from inspect import signature
signature(Graphene)

In [ ]:
signature(cut)

In [ ]:
Graphite.bravais_basis

In [ ]:
Graphite.bravais_basis.append([1/3,1/3,0])

In [ ]:
Graphite.bravais_basis

In [ ]:
gra = Graphite('C', latticeconstant={'a':2.46, 'c':6.70}, size=(1,1,1))
gra_cut = cut(gra, a=(1,0,0), b=(1,2,0), c=(0,0,2.0))
#add_vacuum(gra_cut, 10)
aobj = gra_cut
print(f' Unit cell: a = ({aobj.cell[0,0]:9.5f}, {aobj.cell[0,1]:9.5f}, {aobj.cell[0,2]:9.5f})')
print(f'            b = ({aobj.cell[1,0]:9.5f}, {aobj.cell[1,1]:9.5f}, {aobj.cell[1,2]:9.5f})')
print(f'            c = ({aobj.cell[2,0]:9.5f}, {aobj.cell[2,1]:9.5f}, {aobj.cell[2,2]:9.5f})')
print(f' Number of atoms: {len(aobj.positions):5d}')
print(f' Species, Positions:')
for i in range(len(aobj.positions)):
    print(f'  \'{aobj.symbols[i]:<2}\' ({aobj.positions[i,0]:9.5f}, {aobj.positions[i,1]:9.5f}, {aobj.positions[i,2]:9.5f})')

In [ ]:
fig, ax = plt.subplots(2,1, figsize=(12, 6))
ax[0].set_axis_off()
ax[1].set_axis_off()
plot_atoms(gra_cut, ax[0], radii=1.2, rotation=('0x,0y,0z'))
plot_atoms(gra_cut, ax[1], radii=1.2, rotation=('90x,90y,90z'))
plt.show()

In [ ]:
view(gra_cut, viewer='ngl')

In [ ]:
Graphene.bravais_basis

In [ ]:
Graphene.bravais_basis.append([1/3, 2/3, 1/3])
Graphene.bravais_basis.append([2/3, 1/3, 1/3])
Graphene.bravais_basis.append([2/3, 1/3, 2/3])
Graphene.bravais_basis.append([0, 0, 2/3])

In [ ]:
gra = Graphene('C', latticeconstant={'a':2.46, 'c':6.70+6.7/2}, size=(1,1,1))

In [ ]:
view(gra, viewer='ngl')

In [ ]:
gra_plus = Graphene

In [ ]:
signature(Graphene)

In [ ]:
Graphene.bravais_basis

In [ ]:
del Graphene.bravais_basis[2:5:1]

In [ ]:
Graphene.bravais_basis.pop(2)